In [32]:
print('good to go!')

good to go!


In [2]:
!pip install deepchem


In [3]:
import deepchem as dc
import numpy as np
import pandas as pd
from deepchem.models import Smiles2Vec

# If you haven't installed DeepChem, install it using:
# pip install deepchem


No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
2024-06-25 18:21:47.376198: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/opt/conda/lib/python3.10/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [4]:
import pandas as pd

df_BRD4 = pd.read_parquet('BRD4_data_balanced.parquet')


In [5]:
df_BRD4.shape

(2284820, 2)

In [6]:
train_size = 500_000

In [7]:
# Sample 20% of the dataset
df_BRD4 = df_BRD4[train_size:train_size + train_size]

In [8]:
import pandas as pd
import numpy as np
from rdkit import Chem
import deepchem as dc

# Sample Data
smiles = df_BRD4['molecule_smiles'].tolist()

# Define the character to index mapping, ensuring all unique characters in the SMILES strings are included
unique_chars = sorted(set(''.join(smiles)))
char_to_idx = {char: idx for idx, char in enumerate(unique_chars)}
char_to_idx['PAD'] = len(char_to_idx)  # Add PAD token
char_to_idx['<unk>'] = len(char_to_idx)  # Add unknown token for any out-of-vocabulary characters


# Featurizer
featurizer = dc.feat.SmilesToSeq(char_to_idx, max_len=256, pad_len=0)  # Adjust max_len to 270


In [11]:



# Load the data using CSVLoader with MATFeaturizer
import time
# Measure encoding speed
start_time = time.time()
# Featurize the data
X = featurizer.featurize(smiles)
end_time = time.time()

num_smiles = len(df_BRD4)
encoding_time = end_time - start_time
encoding_speed = num_smiles / encoding_time

print(f"Encoded {num_smiles} SMILES strings in {encoding_time:.2f} seconds.")
print(f"Encoding speed: {encoding_speed:.2f} SMILES strings per second.")


Encoded 500000 SMILES strings in 490.08 seconds.
Encoding speed: 1020.24 SMILES strings per second.


In [12]:
X.shape

(500000, 256)

In [13]:
y = np.array(df_BRD4['binds'])

In [14]:
# Create DeepChem dataset
dataset = dc.data.NumpyDataset(X, y)

In [15]:
# Split the dataset into training and validation sets
splitter = dc.splits.RandomSplitter()
train_dataset, valid_dataset = splitter.train_test_split(dataset, frac_train=0.8)

In [16]:
# Prepare the dataset

# Define the model
model = dc.models.Smiles2Vec(
    char_to_idx=char_to_idx,
    n_tasks=1,
    max_seq_len=256,
    embedding_dim=50,
    n_classes=2,
    use_bidir=True,
    use_conv=True,
    filters=192,
    kernel_size=3,
    strides=1,
    rnn_sizes=[224, 384],
    rnn_types=["GRU", "GRU"],
    mode="classification"
)




In [17]:
# Restore the model from the checkpoint
# Directory where the model is saved
load_dir = 'S2V_model_BRD4_500k'

model.restore(model_dir=load_dir)

# Confirm model is loaded
print("Model loaded successfully from", load_dir)

Model loaded successfully from S2V_model_BRD4_500k


In [18]:
# Custom training loop with verbosity
nb_epoch = 2
for epoch in range(nb_epoch):
    loss = model.fit(train_dataset, nb_epoch=1)
    
    # Check class distribution in training and validation datasets
    train_labels = train_dataset.y
    valid_labels = valid_dataset.y
    
    if len(set(train_labels)) > 1:
        train_score = model.evaluate(train_dataset, [dc.metrics.roc_auc_score])
        print(f"  Train ROC-AUC Score: {train_score['metric-1']}")
    else:
        print(f"  Train ROC-AUC Score: Not defined (only one class present in y_true)")
    
    if len(set(valid_labels)) > 1:
        valid_score = model.evaluate(valid_dataset, [dc.metrics.roc_auc_score])
        print(f"  Valid ROC-AUC Score: {valid_score['metric-1']}")
    else:
        print(f"  Valid ROC-AUC Score: Not defined (only one class present in y_true)")
    
    print(f"Epoch {epoch+1}/{nb_epoch}")
    print(f"  Training Loss: {loss}")

  Train ROC-AUC Score: 0.9549020840640687
  Valid ROC-AUC Score: 0.9556641759062743
Epoch 1/2
  Training Loss: 0.21707298278808593
  Train ROC-AUC Score: 0.9591746811281885
  Valid ROC-AUC Score: 0.959769700834018
Epoch 2/2
  Training Loss: 0.1938931655883789


Epoch 2/2
  Training Loss: 0.2335660743713379


In [19]:
import os

# Directory to save the model
save_dir = 'S2V_model_BRD4_500k_2'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Save the model
model.save_checkpoint(model_dir=save_dir)


In [20]:
# Specify your S3 Bucket and file key
bucket = 'kaggle-leash-bio'
test_parquet_key = 'test.parquet'
test_parquet_location = f's3://{bucket}/{test_parquet_key}'

In [21]:
# Open the Parquet file
df = pd.read_parquet(test_parquet_location, engine='pyarrow')

In [22]:
# Filter for molecules binding with the BRD4 protein
df_BRD4_test = df[df['protein_name'] == 'BRD4']


In [23]:
# # Reduce for quick view of the test data
# df_BRD4_test = df_BRD4_test.sample(frac=0.0001, random_state=42)  # 20% random sample
# print(f"New DataFrame size: {df_BRD4_test.shape}")

In [24]:
# Featurize the data
X_test = featurizer.featurize(df_BRD4_test['molecule_smiles'].tolist())

In [25]:
# Create DeepChem dataset
dataset = dc.data.NumpyDataset(X_test)

In [26]:
# Predict bindings
predictions = model.predict(dataset)

# Extract the probability of the positive class (binding)
probabilities = predictions[:, 0, 1]  # Assuming the second column corresponds to the positive class

In [27]:
# Create resulting DataFrame with 'id' and 'binds' columns
result_df = pd.DataFrame({
    'id': df_BRD4_test['id'],
    'binds': probabilities
})
# Display the resulting DataFrame
result_df

,id,binds
0,295246830,0.011335
3,295246833,0.005345
6,295246836,0.006189
9,295246839,0.005850
12,295246842,0.007947
...,...,...
1674881,296921711,0.031504
1674884,296921714,0.006171
1674887,296921717,0.002856
1674890,296921720,0.008012


In [28]:
len(result_df)

558859

In [29]:
# Assuming result_df is your DataFrame
result_df.to_csv('BRD4_S2VEC_2.csv', index=False)


In [34]:
# Load the prediction CSV files
she_predictions = pd.read_csv('sEH_S2VEC.csv')
hsa_predictions = pd.read_csv('HSA_S2VEC.csv')
brd4_predictions = pd.read_csv('BRD4_S2VEC_2.csv')

# Concatenate the DataFrames
all_predictions = pd.concat([she_predictions, hsa_predictions, brd4_predictions])

# Sort by the 'id' column
all_predictions_sorted = all_predictions.sort_values(by='id')

# Save to a new CSV file
all_predictions_sorted.to_csv('final_submission_S2VEC_2_BRD4.csv', index=False)

print("final_submission.csv created successfully.")

final_submission.csv created successfully.
